# Imports

In [12]:
import datasets
from bs4 import BeautifulSoup
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import re
from evaluation import evaluate

# Constants

In [13]:
PATH_TO_OWL = './LMSS.owl'
LLM_PATH = '../Llama-2-7b-chat-hf'

# Set up Llama

In [14]:
model = AutoModelForCausalLM.from_pretrained(LLM_PATH)
tokenizer = AutoTokenizer.from_pretrained(LLM_PATH)

Loading checkpoint shards: 100%|██████████| 6/6 [00:04<00:00,  1.21it/s]


In [16]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer,torch_dtype=torch.float16,device=1)

OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 1 has a total capacity of 39.43 GiB of which 16.31 MiB is free. Including non-PyTorch memory, this process has 39.41 GiB memory in use. Of the allocated memory 38.04 GiB is allocated by PyTorch, and 56.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:
def get_llama_response(prompt):
    sequences = pipe(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id
        # max_length=256,
        # truncation=True
    )
    return sequences[0]['generated_text']

In [6]:
#example
prompt = '''Write me a poem about Machine Learning.'''
answer = get_llama_response(prompt)
print(answer)

Write me a poem about Machine Learning.

In the depths of code, a world unfolds
Where algorithms dance, and knowledge grows
With each new task, a challenge is posed
To the machines, that learn and know

From the mountains of data, they climb
And find the patterns, that time did line
Their minds aglow, with insights so bright
They solve the problems, of the night

With each new step, they reach the peak
Of knowledge and wisdom, they seek
Their learning curves, a never-ending quest
For the secrets, of the machine's crest

Their minds, a blur of 1s and 0s
A world of ones and zeroes, they know
The language of the digital realm
Where the future, is their domain

So let us marvel, at their might
As they learn and grow, in the light
Of the machines, that learn and know
And the wonders, they bring to show.


# Reading OWL

In [7]:
# Read the contents of the .owl file
with open(PATH_TO_OWL, "r") as owl_file:
    owl_data = owl_file.read()

# Parse the OWL data using BeautifulSoup
soup = BeautifulSoup(owl_data, 'xml')

In [8]:
# Initialize lists to store data
labels = []
definitions = []

# Find all instances of <owl:Class> elements and extract label and definition
for owl_class in soup.find_all('owl:Class'):
    label_element = owl_class.find('rdfs:label')
    definition_element = owl_class.find('skos:definition')
    
    # Check if label and definition elements exist
    if label_element and definition_element:
        label = label_element.text.strip()
        definition = definition_element.text.strip()
        
        # Append data to lists
        labels.append(label)
        definitions.append(definition)

data = {'Label': labels, 'Definition': definitions}
owl_df = pd.DataFrame(data)

owl_df

,Label,Definition
0,Other Personal and Household Goods Repair and ...,See industry description for 811490.
1,Other Converted Paper Product Manufacturing,This industry comprises establishments primari...
2,General Medical and Surgical Hospitals,NULL
3,Confectionery Merchant Wholesalers,This industry comprises establishments primari...
4,Other Specialized Design Services,See industry description for 541490.
...,...,...
14248,Vocational Rehabilitation Services,NULL
14249,Books Printing,This U.S. industry comprises establishments pr...
14250,Petrochemical Manufacturing,See industry description for 325110.
14251,Pesticide and Other Agricultural Chemical Manu...,This industry comprises establishments primari...


### Function to get classes

In [9]:
def filter_label_by_substring(df, substring):
    """
    Filter DataFrame rows containing the specified substring in the 'Label' column
    and return a list of strings in the format "{Label} : {Definition}".
    
    Args:
        df (pandas.DataFrame): Input DataFrame.
        substring (str): Substring to search for.
        
    Returns:
        list: List of strings in the format "{Label} : {Definition}" for matching rows.
    """
    filtered_df = df[df['Label'].str.contains(substring, case=False)]
    output_list = []
    for index, row in filtered_df.iterrows():
        output_list.append(f"{row['Label']} : {row['Definition']}")
    return output_list


In [10]:
#example usage
search_substring = 'hearsay'
result = filter_label_by_substring(owl_df, search_substring)
print(result)

['Motion in Limine to Exclude Hearsay Witness : A Motion in Limine to Exclude Hearsay Witness is a legal request made by one party to prevent the other party from presenting testimony from a witness who will testify about statements made by someone else out of court, which are being offered to prove the truth of the matter asserted.', 'Motion to Exclude Hearsay Witness : A Motion to Exclude Hearsay Witness is a legal request to prevent a witness from testifying in court based on the fact that their testimony is based on hearsay evidence.']


# Dataset

In [11]:
dataset_hearsay = datasets.load_dataset("nguha/legalbench", "hearsay")

/home/ojasva20318/.local/lib/python3.8/site-packages/datasets/load.py:1454: FutureWarning: The repository for nguha/legalbench contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nguha/legalbench
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [12]:
test_df = dataset_hearsay['test'].to_pandas()
test_df

,answer,index,slice,text
0,No,0,Non-assertive conduct,On the issue of whether James is an smart indi...
1,No,1,Non-assertive conduct,On the issue of whether Robert negligently dro...
2,No,2,Non-assertive conduct,On the issue of whether John knew about the co...
3,No,3,Non-assertive conduct,On the issue of whether Michael was guilty of ...
4,No,4,Non-assertive conduct,On the issue of whether William was loved by h...
...,...,...,...,...
89,No,89,Not introduced to prove truth,To prove that Arthur believed that Amy and Dan...
90,No,90,Not introduced to prove truth,To prove that the trademarks of restaurant A a...
91,No,91,Not introduced to prove truth,To prove that Michael knew of the existing pat...
92,No,92,Not introduced to prove truth,To prove that Arthur and Mary had a conversati...


In [13]:
prompts = test_df["text"].tolist()
prompts

['On the issue of whether James is an smart individual, the fact that James came first in his class in law school.',
 'On the issue of whether Robert negligently drove, the fact that Robert fell asleep while driving.',
 'On the issue of whether John knew about the conspiracy, the fact that John likes sweatpants.',
 'On the issue of whether Michael was guilty of murder, the fact that Michael left the crime scene immediately.',
 'On the issue of whether William was loved by his community, the fact that he was selected to speak at his graduation.',
 'On the issue of whether Mary robbed the bank, the fact that Mary went to the bank in disguise.',
 'On the issue of whether Patricia was a fan of Coldplay, the fact that she had a poster with the lyrics of "Viva la Vida" on her bedroom wall.',
 "On the issue of whether Jennifer suffered reputational harm from Linda's article, the fact that Linda worked with several different editors to proof read and cross-check her article.",
 "On the issue o

In [14]:
prompts[0]

'On the issue of whether James is an smart individual, the fact that James came first in his class in law school.'

In [73]:
def add_labels_and_definitions_to_prompt(prompt_text, filtered_labels):
    """
    Add filtered labels and definitions to the prompt.

    Args:
        prompt_text (str): The prompt text.
        filtered_labels (list): List of strings containing labels and definitions.

    Returns:
        str: The full prompt text with filtered labels and definitions added.
    """
    # Initialize full_prompt with prompt_text
    full_prompt = f"""
    Statement : {prompt_text}
    Question: Consider utilizing the following legal ontology classes to frame your argument:

    """
    
    # Add filtered labels and definitions to the prompt
    for label_definition in filtered_labels:
        full_prompt += f"\n{label_definition}"

    # Add the remaining part of the prompt
    full_prompt += """
    Use these ontology classes to structure your argument and analyze whether the information provided falls under the category of hearsay.
    
    Output Format: First word of your answer should be either Yes or No followed by step-by-step reasoning

    Answer: 
    """

    return full_prompt


# Testing - Hearsay

In [60]:
filtered_labels = filter_label_by_substring(owl_df, "hearsay")
filtered_labels

['Motion in Limine to Exclude Hearsay Witness : A Motion in Limine to Exclude Hearsay Witness is a legal request made by one party to prevent the other party from presenting testimony from a witness who will testify about statements made by someone else out of court, which are being offered to prove the truth of the matter asserted.',
 'Motion to Exclude Hearsay Witness : A Motion to Exclude Hearsay Witness is a legal request to prevent a witness from testifying in court based on the fact that their testimony is based on hearsay evidence.']

In [74]:
prompt = add_labels_and_definitions_to_prompt(prompts[0],filtered_labels)

response = get_llama_response(prompt)

print(response)


    Statement : On the issue of whether James is an smart individual, the fact that James came first in his class in law school.
    Question: Consider utilizing the following legal ontology classes to frame your argument:

    
Motion in Limine to Exclude Hearsay Witness : A Motion in Limine to Exclude Hearsay Witness is a legal request made by one party to prevent the other party from presenting testimony from a witness who will testify about statements made by someone else out of court, which are being offered to prove the truth of the matter asserted.
Motion to Exclude Hearsay Witness : A Motion to Exclude Hearsay Witness is a legal request to prevent a witness from testifying in court based on the fact that their testimony is based on hearsay evidence.
    Use these ontology classes to structure your argument and analyze whether the information provided falls under the category of hearsay.
    
    Output Format: First word of your answer should be either Yes or No followed by st

In [34]:
responses = []

filtered_labels = filter_label_by_substring(owl_df, "hearsay")

for i, prompt_text in enumerate(prompts):

    full_prompt = add_labels_and_definitions_to_prompt(prompt_text, filtered_labels)

    response = get_llama_response(full_prompt)
    
    responses.append(response)
    
    print(f"Done for prompt {i+1}")
    # print(response)


/home/ojasva20318/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Done for prompt 1
Done for prompt 2
Done for prompt 3
Done for prompt 4
Done for prompt 5
Done for prompt 6
Done for prompt 7
Done for prompt 8
Done for prompt 9
Done for prompt 10
Done for prompt 11
Done for prompt 12
Done for prompt 13
Done for prompt 14
Done for prompt 15
Done for prompt 16
Done for prompt 17
Done for prompt 18
Done for prompt 19
Done for prompt 20
Done for prompt 21
Done for prompt 22
Done for prompt 23
Done for prompt 24
Done for prompt 25
Done for prompt 26
Done for prompt 27
Done for prompt 28
Done for prompt 29
Done for prompt 30
Done for prompt 31
Done for prompt 32
Done for prompt 33
Done for prompt 34
Done for prompt 35
Done for prompt 36
Done for prompt 37
Done for prompt 38
Done for prompt 39
Done for prompt 40
Done for prompt 41
Done for prompt 42
Done for prompt 43
Done for prompt 44
Done for prompt 45
Done for prompt 46
Done for prompt 47
Done for prompt 48
Done for prompt 49
Done for prompt 50
Done for prompt 51
Done for prompt 52
Done for prompt 53
Do

In [49]:
print(responses[50])


    Statement : Alex is being prosecuted for participation in a criminal conspiracy. To prove that Alex participated in the conspiracy, the prosecution's witness testifies that she heard Alex making plans to meet with his co-conspirators.
    Question: Consider utilizing the following legal ontology classes to frame your argument:

    
Motion in Limine to Exclude Hearsay Witness : A Motion in Limine to Exclude Hearsay Witness is a legal request made by one party to prevent the other party from presenting testimony from a witness who will testify about statements made by someone else out of court, which are being offered to prove the truth of the matter asserted.
Motion to Exclude Hearsay Witness : A Motion to Exclude Hearsay Witness is a legal request to prevent a witness from testifying in court based on the fact that their testimony is based on hearsay evidence.
    Use these ontology classes to structure your argument and analyze whether the information provided falls under the ca

### Parsing output

In [39]:
# Example response
response = """
Statement : On the issue of the faultiness of the designed house, the drawing the witness made on the stand during testimony.
    Question: Consider utilizing the following legal ontology classes to frame your argument:

    
Motion in Limine to Exclude Hearsay Witness : A Motion in Limine to Exclude Hearsay Witness is a legal request made by one party to prevent the other party from presenting testimony from a witness who will testify about statements made by someone else out of court, which are being offered to prove the truth of the matter asserted.
Motion to Exclude Hearsay Witness : A Motion to Exclude Hearsay Witness is a legal request to prevent a witness from testifying in court based on the fact that their testimony is based on hearsay evidence.
    Use these ontology classes to structure your argument and analyze whether the information provided falls under the category of hearsay.
    
    Output Format: First word of your answer should be Yes or No
    Answer:
    
    Yes, the information provided falls under the category of hearsay.
    Reasoning:
    The witness testimony provided is a statement made by someone else outside of court, which is being offered to prove the truth of the matter asserted. This is a classic example of hearsay evidence, which is not admissible in court.
    Therefore, the motion in limine to exclude hearsay witness should be granted.

Explanation:
The provided information falls under the category of hearsay because it is a statement made by someone else outside of court, which is being offered to prove the truth of the matter asserted. Hearsay evidence is not admissible in court, and the witness testimony provided is a clear example of hearsay. Therefore, the motion in limine to exclude hearsay witness should be granted.

Please let me know if you need any further assistance
"""

# Use regex to extract the answer
match = re.search(r"Answer:\s+(Yes|No)", response)
if match:
    answer = match.group(1)
else:
    answer = "Unknown"

print(answer)  # Output: "Yes"

Yes


In [40]:
def extract_answers(original_responses):
    """
    Extracts "Yes" or "No" from each response in the original_responses list.
    
    Args:
        original_responses (list): List of strings containing responses.
        
    Returns:
        list: List containing "Yes" or "No" extracted from each response.
    """
    # Create an empty list to store extracted answers
    extracted_answers = []

    # Process each response
    for response in original_responses:
        # Use regex to extract the answer
        match = re.search(r"Answer:\s+(Yes|No)", response)
        if match:
            answer = match.group(1)
        else:
            answer = "Unknown"

        # Append the extracted answer to the list
        extracted_answers.append(answer)

    return extracted_answers

In [41]:
extracted_answers = extract_answers(responses)

In [42]:
print(extracted_answers)

['Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes']


In [45]:
evaluate("hearsay", extracted_answers, test_df["answer"].tolist()[:len(extracted_answers)])

0.5